In [1]:
import urllib.request

In [2]:
destination = 'сертификат'
url = 'https://storage.yandexcloud.net/cloud-certs/CA.pem'
urllib.request.urlretrieve(url, destination)

('сертификат', <http.client.HTTPMessage at 0x7f5e7467bca0>)

In [3]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Анализ базы данных сервиса для чтения книг по подписке

<div style="border:solid green 2px; padding: 20px">

<font size = 5> ***Цель исследования:*** </font>

<font size = 4> Получить данные для создания ценностного предложения для нового продукта.  </font>

<div style="border:solid green 2px; padding: 20px">
    
<font size = 5> **Содержание:**</font>
    
* **1 [Исследование таблиц](#1)**
* **2 [Количество книг, изданных после 01 января 2000](#2)**
* **3 [Общее количество обзоров и средняя оценка для каждой книги](#3)**
* **4 [Издательство, выпустившее наибольшее количество книг (толще 50 страниц)](#4)**
* **5 [Автор с самой высокой средней оценкой книг (среди книг, имеющих 50 и более отзывов) ](#5)**
* **6 [Среднее количество обзоров от пользователей, поставивших более 50 оценок](#6)**
* **7 [Определим авторов с наименьшим средним промежутком времени между публикациями книг, с использованием "оконных функций"](#7) **     
* **8 [Выводы](#8)**


<a id='1'></a>
##  Исследование таблиц

In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<a id='2'></a>
##  Количество книг, изданных после 01 января 2000

In [9]:
query = '''
SELECT COUNT(DISTINCT(book_id)) AS "кол-во книг"
FROM books
WHERE EXTRACT (YEAR FROM CAST(publication_date AS timestamp)) > 1999
'''
pd.io.sql.read_sql(query, con = engine)

,кол-во книг
0,821


**Вывод:** Количество книг, изданных с 2000 года: 821

<a id='3'></a>
##  Общее количество обзоров и средняя оценка для каждой книги

In [10]:
query = '''
SELECT b.title AS "книга",
       COUNT(DISTINCT(re.review_id)) AS "кол-во отзывов",
       round(AVG(ra.rating),1) AS "средняя оценка"
FROM books AS b
LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
LEFT JOIN reviews AS re ON b.book_id=re.book_id
GROUP BY b.book_id
ORDER BY "кол-во отзывов" DESC
'''
pd.io.sql.read_sql(query, con = engine)






,книга,кол-во отзывов,средняя оценка
0,Twilight (Twilight #1),7,3.7
1,Water for Elephants,6,4.0
2,The Glass Castle,6,4.2
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,The Curious Incident of the Dog in the Night-Time,6,4.1
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7
996,The Natural Way to Draw,0,3.0
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.0
998,Essential Tales and Poems,0,4.0


**Комментарий:** Проверим все ли книги вошли в таблицу. 

In [11]:
query = '''
SELECT COUNT(DISTINCT(book_id)) AS "кол-во книг всего"
FROM books
'''
pd.io.sql.read_sql(query, con = engine)

,кол-во книг всего
0,1000


**Комментарий:** Все книги, представленные в базе данных, вошли в таблицу. 

**Вывод:** Какой-либо закономерности в рейтинге книги по отзывам покупателей и кол-ве отзывов по полученной таблице проследить не удалось.  

<a id='4'></a>
##  Издательство, выпустившее наибольшее количество книг (толще 50 страниц)

In [12]:
query = '''
SELECT p.publisher AS "самое мощное издательство",
       COUNT(DISTINCT(b.book_id)) AS "кол-во книг"
FROM publishers AS p
RIGHT JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE b.num_pages>50
GROUP BY p.publisher
ORDER BY "кол-во книг" DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)


,самое мощное издательство,кол-во книг
0,Penguin Books,42


<a id='5'></a>
##  Автор с самой высокой средней оценкой книг (среди книг, имеющих 50 и более отзывов

In [13]:
query = '''
WITH
xas AS (SELECT book_id,
              COUNT(DISTINCT(rating_id)) AS "кол-во оценок"
       FROM ratings
       GROUP BY book_id
       HAVING COUNT(DISTINCT(rating_id))>=50)

SELECT a.author AS "автор",
       round(AVG(ra.rating), 2) AS "средняя оценка"
FROM authors AS a
JOIN books AS b ON a.author_id=b.author_id
JOIN ratings AS ra ON ra.book_id=b.book_id
JOIN xas ON xas.book_id=b.book_id
GROUP BY a.author
ORDER BY "средняя оценка" DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,автор,средняя оценка
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** Самый популярный автор - J.K. Rowling/Mary GrandPré (среднняя оценка 4,3)

<a id='6'></a>
##  Среднее количество обзоров от пользователей, поставивших более 50 оценок

In [14]:
query = '''
WITH
rat AS (SELECT username,
               COUNT(rating_id)
       FROM ratings
       GROUP BY username
       HAVING COUNT(rating_id)>50), 
       
rev AS (SELECT username,
               COUNT(review_id) AS "отзывы"
        FROM reviews
        GROUP BY username)       
       
SELECT round(AVG(rev.отзывы), 0) AS "ср. кол-во отзывов акт-ых польз." 
FROM rat
INNER JOIN rev ON rat.username=rev.username 
'''
pd.io.sql.read_sql(query, con = engine)

,ср. кол-во отзывов акт-ых польз.
0,24.0


**Вывод:** Среднее количество отзывов активных пользователей (поставивших более 50 оценок) - 24 отзыва

<a id='7'></a>
##  Определим авторов с наименьшим средним промежутком времени между публикациями книг  

In [15]:
query = '''
WITH publications AS 
       (SELECT author_id,
               publication_date AS last_publication,
               LAG(publication_date) OVER (PARTITION BY author_id ORDER BY publication_date) AS previous_publication,
               ((publication_date-LAG(publication_date) OVER (PARTITION BY author_id ORDER BY publication_date))/30) AS months_between
                              
        FROM books)
SELECT a.author AS автор,
       ROUND(AVG(p.months_between)) AS количество_месяцев
FROM authors AS a
LEFT JOIN publications AS p ON p.author_id=a.author_id
WHERE p.months_between>0
GROUP BY a.author
ORDER BY AVG(p.months_between)

'''
pd.io.sql.read_sql(query, con = engine)

,автор,количество_месяцев
0,Rick Riordan,1.0
1,Tad Williams,1.0
2,Nora Roberts,1.0
3,Carl Hiaasen,1.0
4,Dave Barry/Ridley Pearson/Greg Call,2.0
...,...,...
118,Doris Kearns Goodwin,133.0
119,John Sandford,139.0
120,Robin McKinley,139.0
121,Bret Easton Ellis,145.0


**Вывод:** Средний промежуток времени между публикациями книг следующих авторов составляет менее 2х месяцев:
* Rick Riordan
* Tad Williams
* Nora Roberts
* Carl Hiaasen 

<a id='8'></a>
##  Выводы

* Более 82% книг, представленных в приложении выпущены начиная с 2000 года. 
* Книга с наибольшим количеством отзывов "Сумерки" - 7 отзывов, средняя оценка 3,7. При этом какая-либо закономерность в распределении рейтинга и количства отзывов не прослеживается. 
* Издательство, выпустившее наибольшее количество книг, представленных в приложении Penguin Books - 42 книги. 
* Самый популярный автор, среди книг, имеющих более 50 оценок - J.K. Rowling/Mary GrandPré средняя оценка 4,3
* Активные покупатели (поставившие более 50 оценок) в среднем оставляют 24 отзыва. 